# Setup

In this notebook, we will demonstrate various sampling methods in Pandas and Dask. To illustrate the methods, we use a dataset on the [annual number of objects launched into space from Our World in Data](https://ourworldindata.org/grapher/yearly-number-of-objects-launched-into-outer-space) and hosted in [Tidy Tuesday's Repository](https://github.com/rfordatascience/tidytuesday/blob/main/data/2024/2024-04-23/readme.md).

In [ ]:
%load_ext dotenv
%dotenv 
%run update_path.py

In [ ]:
import dask.dataframe as dd
from glob import glob
from utils.logger import get_logger
_logs = get_logger(__name__)

In [ ]:
import pandas as pd
outer_space_dt = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2024/2024-04-23/outer_space_objects.csv')

In [ ]:
outer_space_dt.info()

In [ ]:
idx = outer_space_dt['Year'] >= 2020
idx &= outer_space_dt['Entity'] != 'World'
outer_space_dt = outer_space_dt[idx]

# Sampling in Python

There are different packages that allow sampling. A practical approach is to use pandas/Dask sampling methods.

## Random Sampling

Sample n rows from a dataframe with [`df.sample()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html).

```python
DataFrame.sample(
    n=None, frac=None, replace=False, weights=None, 
    random_state=None, axis=None, ignore_index=False
    )
```

In [ ]:
outer_space_dt.sample(n = 10, random_state = 42)

In [ ]:
import random
random.seed(42)
frac = 0.5

simple_sample_dt = outer_space_dt.sample(frac = frac)
simple_sample_dt.shape, outer_space_dt.shape

## Stratified Sampling

Use `groupby()` and `.sample()` for stratified sampling.

In [ ]:
strat_sample_dt=outer_space_dt.groupby('Entity').sample(frac=frac, random_state=42)
strat_sample_dt.shape, outer_space_dt.shape

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Prepare data for comparison
df_orig = outer_space_dt['Entity'].value_counts().reset_index()
df_orig.columns = ['Entity', 'count']
df_orig['sample_type'] = 'Original'

df_simple = simple_sample_dt['Entity'].value_counts().reset_index()
df_simple.columns = ['Entity', 'count']
df_simple['sample_type'] = 'Simple Random'

df_strat = strat_sample_dt['Entity'].value_counts().reset_index()
df_strat.columns = ['Entity', 'count']
df_strat['sample_type'] = 'Stratified'

# Combine all data
combined_df = pd.concat([df_orig, df_simple, df_strat])

# Create faceted plot
sns.set_style("whitegrid")
g = sns.catplot(data=combined_df, x='Entity', y='count', col='sample_type', 
                kind='bar', height=5, aspect=1, palette='Set2')
g.set_xticklabels(rotation=90, ha='right', fontsize=5)
g.set_titles("{col_name}")
plt.tight_layout()
plt.show()

# Sampling in Dask

Stratified sampling in Dask works somewhat differently. The code below will raise a Key Error (the "key" *sample* is not found).

```python
strat_sample_dd = (dd_dt.groupby('Entity', group_keys=False)
                        .sample(frac = frac)
                        .compute())
```

However, stratified sampling in Dask can be done with `groupby().apply()` and a lambda function.

In [ ]:
dd_dt = dd.from_pandas(outer_space_dt, npartitions=4)

strat_sample_dd = (dd_dt
                      .groupby('Entity', group_keys=False)
                      .apply(lambda x: x.sample(frac = frac))
                      .compute()
                      .reset_index())
